In [114]:
#importação de pacotes

import pandas as pd
import numpy as np
import scipy
import nltk

In [115]:
df = pd.read_csv(r'C:\Users\arquivodeinteresse.csv', sep = ',', low_memory = False)

In [116]:
df.head()

,Review Text,Sentiment
0,NaN,Positivo
1,NaN,Positivo
2,Boa,Positivo
3,"Não consigo gerar a fatura em PDF, no mesmo fo...",Negativo
4,NaN,Positivo


In [117]:
df.tail()

,Review Text,Sentiment
5674,"Baixei a última versão recentemente, ainda não...",Negativo
5675,NaN,Positivo
5676,NaN,Positivo
5677,"Gostei muito prático, descomplicado",Positivo
5678,Vive parando de funcionar,Negativo


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5679 entries, 0 to 5678
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  2756 non-null   object
 1   Sentiment    5679 non-null   object
dtypes: object(2)
memory usage: 88.9+ KB


In [119]:
df.describe()

,Review Text,Sentiment
count,2756,5679
unique,2133,2
top,Muito bom,Positivo
freq,96,5144


In [120]:
df['Review Text'] = df['Review Text'].replace(np.nan, 'Sem comentários')
df.head()

,Review Text,Sentiment
0,Sem comentários,Positivo
1,Sem comentários,Positivo
2,Boa,Positivo
3,"Não consigo gerar a fatura em PDF, no mesmo fo...",Negativo
4,Sem comentários,Positivo


In [121]:
indexName = df[df['Review Text'] == 'Sem comentários'].index
df.drop(indexName, inplace = True)
df.head()

,Review Text,Sentiment
2,Boa,Positivo
3,"Não consigo gerar a fatura em PDF, no mesmo fo...",Negativo
5,Atende bem!,Positivo
7,Rápido e prático,Positivo
8,Rapidez,Positivo


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2755 entries, 2 to 5678
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  2755 non-null   object
 1   Sentiment    2755 non-null   object
dtypes: object(2)
memory usage: 64.6+ KB


In [123]:
#Alterando o nome dos cabeçalhos do Dataframe

df.rename(columns={
                   'Review Text': 'Comentario',
                   'Sentiment': 'Sentimento',
                  }, inplace=True)

df.head()

,Comentario,Sentimento
2,Boa,Positivo
3,"Não consigo gerar a fatura em PDF, no mesmo fo...",Negativo
5,Atende bem!,Positivo
7,Rápido e prático,Positivo
8,Rapidez,Positivo


In [124]:
df = df.reset_index(drop=True)


In [125]:
df

,Comentario,Sentimento
0,Boa,Positivo
1,"Não consigo gerar a fatura em PDF, no mesmo fo...",Negativo
2,Atende bem!,Positivo
3,Rápido e prático,Positivo
4,Rapidez,Positivo
...,...,...
2750,Prático e rápido.,Positivo
2751,Sempre nota 10,Positivo
2752,"Baixei a última versão recentemente, ainda não...",Negativo
2753,"Gostei muito prático, descomplicado",Positivo


In [126]:
df.Sentimento.value_counts()

Positivo    2317
Negativo     438
Name: Sentimento, dtype: int64

In [127]:
print((df.Sentimento.value_counts()/df.shape[0])*100)

Positivo    84.101633
Negativo    15.898367
Name: Sentimento, dtype: float64


In [128]:
#Transformando os sentimentos em valores numéricos

df.Sentimento = df['Sentimento'].map({'Positivo': 1, 'Negativo': 0})
df.head()

,Comentario,Sentimento
0,Boa,1
1,"Não consigo gerar a fatura em PDF, no mesmo fo...",0
2,Atende bem!,1
3,Rápido e prático,1
4,Rapidez,1


In [129]:
#Importação do pacote sklearn

import sklearn
sklearn.__version__

# Importando o módulo Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer

#Vetorização
vect = CountVectorizer(ngram_range=(1, 1))
vect.fit(df.Comentario)
text_vect = vect.transform(df.Comentario)

In [130]:
from sklearn.model_selection import train_test_split

In [131]:
#Dividindo os dados em teste e treino

X_train, X_test, y_train, y_test = train_test_split (text_vect, df.Sentimento, test_size = 0.30, random_state = 42)

In [132]:
y_test.value_counts()

1    712
0    115
Name: Sentimento, dtype: int64

In [133]:
print((y_test.value_counts()/y_test.shape[0])*100)

1    86.094317
0    13.905683
Name: Sentimento, dtype: float64


# Logistic Regression

In [134]:
from sklearn.linear_model import LogisticRegression


In [135]:
modeloLR = LogisticRegression(random_state=0, solver='newton-cg')
modeloLR = modeloLR.fit(X_train, y_train)

In [136]:
from sklearn import metrics

In [137]:
y_prediction = modeloLR.predict(X_test)

In [138]:
#Expondo as métricas do modelo

print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(y_test, y_prediction, labels = [1,0])))
print("")

print("Classification Report")
print(metrics.classification_report(y_test, y_prediction, labels = [1,0]))

Confusion Matrix
[[683  29]
 [ 59  56]]

Classification Report
              precision    recall  f1-score   support

           1       0.92      0.96      0.94       712
           0       0.66      0.49      0.56       115

    accuracy                           0.89       827
   macro avg       0.79      0.72      0.75       827
weighted avg       0.88      0.89      0.89       827



# Random Forest

In [139]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
modeloRF = RandomForestClassifier(random_state = 42)
modeloRF.fit(X_train, y_train.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [141]:
rf_predict = modeloRF.predict(X_test)

In [142]:
#Expondo as métricas do modelo

print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(y_test, rf_predict, labels = [1,0])))
print("")

print("Classification Report")
print(metrics.classification_report(y_test, rf_predict, labels = [1,0]))

Confusion Matrix
[[690  22]
 [ 69  46]]

Classification Report
              precision    recall  f1-score   support

           1       0.91      0.97      0.94       712
           0       0.68      0.40      0.50       115

    accuracy                           0.89       827
   macro avg       0.79      0.68      0.72       827
weighted avg       0.88      0.89      0.88       827

